In [59]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ta

# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

API_KEY = None
SECRET_KEY = None

import os
from dotenv import load_dotenv

load_dotenv(override=True)

if API_KEY is None:
    API_KEY = os.environ.get('ALP_API_KEY')

if SECRET_KEY is None:
    SECRET_KEY = os.environ.get('ALP_SEC_KEY')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)

from alpaca.data.enums import Adjustment

In [61]:
from regime_tickers import custom_vol_subset, custom_exp_subset

In [62]:
stock_historical_data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# alpaca has no older data than 2016-01-04 for this symbol set
earliest_date = datetime(2016, 1, 4, tzinfo=ZoneInfo('America/New_York'))
# earliest_timestamps_by_symbol = df_adj.reset_index().groupby('symbol')['timestamp'].min()

req = StockBarsRequest(
    symbol_or_symbols = list(set(custom_vol_subset + custom_exp_subset + ['SPY'])),  # add SPY for market reference
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Day), 
    start = earliest_date,                    
    limit = None,    
    adjustment=Adjustment('all') # adjust for splits and dividends                                           
)
df_adj = stock_historical_data_client.get_stock_bars(req).df.reset_index().set_index('timestamp')
df_adj = df_adj.sort_values(by=['symbol', 'timestamp']) # Ensure sorted for correct rolling calcs


In [63]:
pd.set_option('display.max_rows', 300)
print("Total NANs:", df_adj.pivot(columns="symbol").isna().sum().sum())
print(df_adj.pivot(columns="symbol").isna().sum())

pd.reset_option('display.max_rows')

Total NANs: 0
             symbol
open         BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
high         BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
low          BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
close        BND       0
             CPER      0
             GLD       0
             HYG       0
             IWM       0
             SCHP      0
             SPY       0
             VIXY      0
             XLI       0
volume       BND       0
             CPER      0
             GLD

In [64]:
############## windows = [5, 20, 60]
windows = [20]

In [65]:
# Calculate Daily Log Returns
df_adj['log_returns'] = df_adj.groupby('symbol')['close'].transform(lambda x: np.log(x / x.shift(1)))

In [66]:
z_window = 60

In [67]:
# Rolling Standard Deviation (Volatility). For further computations, later we will use z-scores of these values only
for w in windows:
    df_adj[f'rolling_std_20'] = df_adj.groupby('symbol')['log_returns'].transform(
        lambda x: x.rolling(window=w, min_periods=w).std()
    )

    df_adj[f'z_rolling_std_20'] = df_adj.groupby('symbol')[f'rolling_std_20'].transform(
        lambda x: (x - x.rolling(z_window).mean()) / x.rolling(z_window).std()
    )

In [68]:
# Range Ratio (High - Low) / Close, per asset
df_adj['range_ratio'] = (df_adj['high'] - df_adj['low']) / df_adj['close']
# Adding smoothing (e.g., rolling average of 5 days)
df_adj['range_ratio_smooth'] = df_adj.groupby('symbol')['range_ratio'].transform(lambda x: x.rolling(5).mean())
# Standardizing with Z-scores for regime shift detection
df_adj['z_range_ratio_smooth'] = df_adj.groupby('symbol')['range_ratio_smooth'].transform(lambda x: (x - x.rolling(z_window).mean()) / x.rolling(z_window).std())

df_adj.drop(columns=['range_ratio', 'range_ratio_smooth'], inplace=True)

In [69]:
# Volatility Shock Count
# The number of days in a recent lookback window where daily returns exceeded 2 standard deviations, either up or down.
# It captures the frequency of abnormal moves — a key indicator for risk, panic clusters, or momentum bursts.

# 1. Calculate return z-scores per symbol
df_adj['z_log_returns'] = df_adj.groupby('symbol')['log_returns'].transform(
    lambda x: (x - x.rolling(60).mean()) / x.rolling(60).std()
)

# 2. Count shocks in rolling window (e.g., abs(z) > 2 over last 20 days)
df_adj['vol_shock_count_20'] = df_adj.groupby('symbol')['z_log_returns'].transform(
    lambda x: x.rolling(20).apply(lambda r: (abs(r) > 2).sum(), raw=True)
)

df_adj.drop(columns=['log_returns'], inplace=True)

In [70]:
df_adj = df_adj.sort_values(by=['symbol', 'timestamp'])

In [71]:
df_pivot = df_adj.pivot(columns="symbol")
# Flatten columns ('close', 'QQQ') -> 'close_QQQ'
df_pivot.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivot.columns]

In [72]:
# Volatility Cross-Asset Dispersion lets you measure how much the volatility levels themselves are diverging across assets. It's like saying: are
#  all sectors jittery together or are some calm while others are chaotic? Great for reading the market’s internal stress or divergence.

# Step 1: Filter only the 'rolling_std_20' columns
rolling_std_cols = [col for col in df_pivot.columns if col.startswith('rolling_std_20_')]

# Step 2: Compute cross-sectional std (dispersion) across symbols for each timestamp
vol_dispersion = df_pivot[rolling_std_cols].std(axis=1)

# Step 3: Add back into df_pivot as a new column
df_pivot['volatility_dispersion'] = vol_dispersion

In [73]:
# Realized Vol / Implied Vol compares actual historical price movement (realized vol) with what the market expects (implied vol). 
# A ratio >1 means realized volatility has exceeded market expectations — a sign of surprise, dislocation, or a catch-up in pricing. 
# A ratio <1 suggests implied vol is elevated — often in times of uncertainty or hedging demand.

# Z-realized vs Implied Volatility Ratios

# Reference realized vol
realized = df_pivot['rolling_std_20_IWM']

# Loop over each implied asset and compute z-scored realized/implied ratios
implied = df_pivot['rolling_std_20_VIXY']
ratio = realized / implied
z_col = f'z_realized_implied_vixy'

df_pivot[z_col] = (ratio - ratio.rolling(z_window).mean()) / ratio.rolling(z_window).std()


In [74]:
# Volatility Spread = VIXY implied vol proxy – realized vol of SPY/IWM
# Z-score Level	What It Means
# > +2	Implied vol (VIXY) is much higher than realized vol → Fear spike
# < -2	Realized vol is unusually high vs what’s priced in → Complacency mispricing?
# Around 0	Implied and realized vol are in sync → Stable regime

# Step 1: Compute average realized vol of SPY and IWM
realized_avg = df_pivot[['rolling_std_20_SPY', 'rolling_std_20_IWM']].mean(axis=1)

# Step 2–4: Loop over implied assets to compute z-scored spread and clean up
spread = df_pivot['rolling_std_20_VIXY'] - realized_avg
z_col = f'z_vol_spread_vixy'

df_pivot[z_col] = (spread - spread.rolling(z_window).mean()) / spread.rolling(z_window).std()

In [75]:
# Volatility Skew = Difference in realized vol between two symbols
# IWM vs SPY — Small Caps vs Broad Market - GLD vs SPY — Gold vs Broad Market
# Define the assets to compare against SPY
skew_assets = ['IWM', 'GLD']

# Loop to compute z-scored volatility skew vs SPY
for ticker in skew_assets:
    spread = df_pivot[f'rolling_std_20_{ticker}'] - df_pivot['rolling_std_20_SPY']
    z_col = f'z_vol_skew_{ticker.lower()}'
    
    df_pivot[z_col] = (spread - spread.rolling(z_window).mean()) / spread.rolling(z_window).std()

In [76]:
# Step 1: Identify relevant columns (z-scored rolling std)
z_vol_cols = [col for col in df_pivot.columns if col.startswith('z_rolling_std_20_')]

# Step 2: Compute rolling correlation matrix and extract average pairwise correlation
def compute_volatility_correlation_regime(df, columns, window=20):
    avg_corr = []

    for i in range(window, len(df)):
        window_df = df.iloc[i - window:i][columns]
        corr_matrix = window_df.corr()

        # Extract upper triangle without diagonal
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        mean_corr = upper_triangle.stack().mean()
        avg_corr.append(mean_corr)

    # Align with original index
    avg_corr_series = pd.Series(avg_corr, index=df.index[window:])
    return avg_corr_series

# Step 3: Run the function
correlation_regime_vol = compute_volatility_correlation_regime(df_pivot, z_vol_cols, window=20)

# Step 4: Add to df_pivot (optional)
df_pivot.loc[correlation_regime_vol.index, 'correlation_regime_vol'] = correlation_regime_vol

In [77]:
# Expansion vs. Contraction
# Advance-Decline Ratio (ADR)

# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Calculate previous close using shift
prev_close = df_pivot[close_cols].shift(1)

# Step 3: Calculate change
change = df_pivot[close_cols] - prev_close

# Step 4: Classify movement
advance = (change > 0).astype(int)
decline = (change < 0).astype(int)

# Step 5: Count advances and declines per timestamp
advancing_count = advance.sum(axis=1)
declining_count = decline.sum(axis=1)

# Step 6: Calculate ADR  manual encoding to avoid division by zero

adr_series = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    adv = advancing_count[i]
    dec = declining_count[i]
    
    if adv == 0 and dec == 0: # avoifd division by zero
        adr_series[i] = 1.0  # neutral market
    elif dec == 0:
        adr_series[i] = len(set(custom_vol_subset + custom_exp_subset))  # strong bullish signal
    else:
        adr_series[i] = adv / dec

# analyzing trends, a rolling average helps
adr_smoothed = pd.Series(adr_series).rolling(window=5).mean()

# Step 7: Add ADR to df_pivot
df_pivot['ADR_smooth'] = adr_smoothed


In [78]:
# Expansion vs. Contraction
# Zweig Breadth Thrust (ZBT) is a classic and powerful breadth thrust indicator used to identify the start of strong bullish moves

# Step 1: Use previously computed advance matrix
# (from your ADR calculation)
# advance = (change > 0).astype(int)

# Step 2: Compute percentage of advancing stocks per timestamp
advancing_percent = advance.sum(axis=1) / advance.shape[1]

# Step 3: Compute 10-day moving average of advancing percent, ema more smooth than sma
zbt_series = advancing_percent.ewm(span=10, adjust=False).mean()
df_pivot['zbt'] = zbt_series

In [79]:
# Expansion vs. Contraction
# % tickers Above MA50/MA200 Market-wide participation
# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Compute MA50 and MA200 for each symbol
ma50 = df_pivot[close_cols].rolling(window=50, min_periods=1).mean()
ma200 = df_pivot[close_cols].rolling(window=200, min_periods=1).mean()

# Step 3: Compare close to MA50 and MA200
above_ma50 = (df_pivot[close_cols] > ma50).astype(int)
above_ma200 = (df_pivot[close_cols] > ma200).astype(int)

# Step 4: Compute % of stocks above each MA
pct_above_ma50 = above_ma50.sum(axis=1) / len(close_cols) * 100
pct_above_ma200 = above_ma200.sum(axis=1) / len(close_cols) * 100

# Step 5: Add to df_pivot
df_pivot['Pct_Above_MA50'] = pct_above_ma50
df_pivot['Pct_Above_MA200'] = pct_above_ma200

In [80]:
# Expansion vs. Contraction
# Number of stocks making a new 20-day high, Number of stocks making a new 20-day low, New Highs vs. New Lows Ratio
# Step 1: Identify close columns
close_cols = [col for col in df_pivot.columns if col.startswith('close_')]

# Step 2: Compute rolling 20-day high and low
rolling_high = df_pivot[close_cols].rolling(window=20, min_periods=1).max()
rolling_low = df_pivot[close_cols].rolling(window=20, min_periods=1).min()

# Step 3: Identify new highs and new lows
new_highs = (df_pivot[close_cols] >= rolling_high).astype(int)
new_lows = (df_pivot[close_cols] <= rolling_low).astype(int)

# Step 4: Count per timestamp
nh_count = new_highs.sum(axis=1)
nl_count = new_lows.sum(axis=1)

# Step 5: Compute NH/NL Ratio with manual encoding to avoid division by zero
nh_nl_ratio = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    nh = nh_count[i]
    nl = nl_count[i]
    
    if nl == 0 and nh == 0: # avoifd division by zero
        nh_nl_ratio[i] = 1.0  # flat market
    elif nl == 0: # if there are no new lows, we have a strong bullish signal, avoid division by zero
        nh_nl_ratio[i] = len(set(custom_vol_subset + custom_exp_subset))  # strong bullish signal
    else:
        nh_nl_ratio[i] = nh / nl # normal ratio


nh_nl_ratio_smoothed = pd.Series(nh_nl_ratio).rolling(window=5).mean()

# Step 6: Add to df_pivot
df_pivot['NH_NL_Ratio_smooth'] = nh_nl_ratio_smoothed

In [81]:
# Expansion vs. Contraction
# Volume Surge feature, which compares total volume at each timestamp to its rolling average
# Step 1: Identify volume columns
volume_cols = [col for col in df_pivot.columns if col.startswith('volume_')]

# Step 2: Compute total volume per timestamp
total_volume = df_pivot[volume_cols].sum(axis=1)

# Step 3: Compute rolling average volume (e.g., 20-day)
rolling_avg_volume = total_volume.rolling(window=20, min_periods=1).mean()

# Step 4: Compute volume surge ratio
volume_surge_ratio = total_volume / rolling_avg_volume

# Step 5: Manual encoding to avoid NaN or inf
volume_surge = pd.Series(index=df_pivot.index, dtype=float)

for i in df_pivot.index:
    tv = total_volume[i]
    rv = rolling_avg_volume[i]
    
    if rv == 0:
        volume_surge[i] = 1.0  # neutral if no prior volume
    else:
        volume_surge[i] = tv / rv

df_pivot['volum_surge'] = volume_surge

In [82]:
# Drop 'SPY' columns to avoid redundancy, they where just used for comparison
df_pivot.drop(columns=[col for col in df_pivot.columns if 'SPY' in col], inplace=True)

# Drop 'rolling_std_20' columns to avoid redundancy, they where just used for comparison, we keep z-scores only
# Drop columns that start with 'rolling_std_20'
df_pivot.drop(columns=[col for col in df_pivot.columns if col.startswith("rolling_std_20")], inplace=True)

In [83]:
# after 2*max(windows) +z_window +1 days, not nans
df_pivot_clean = df_pivot[max(windows)+z_window+1:]

print(df_pivot_clean.isna().sum().sum()) 

0


In [84]:
df_stationary = df_pivot_clean.drop(
    columns=[col for col in df_pivot_clean.columns if any(x in col for x in ["vwap", "low", "high", "close", "open", "volume",
                                                                              "trade_count", "z_log_returns", "z_range_ratio_smooth", "z_rolling_std"])]
)

In [85]:
df_stationary.columns

Index(['vol_shock_count_20_BND', 'vol_shock_count_20_CPER',
       'vol_shock_count_20_GLD', 'vol_shock_count_20_HYG',
       'vol_shock_count_20_IWM', 'vol_shock_count_20_SCHP',
       'vol_shock_count_20_VIXY', 'vol_shock_count_20_XLI',
       'volatility_dispersion', 'z_realized_implied_vixy', 'z_vol_spread_vixy',
       'z_vol_skew_iwm', 'z_vol_skew_gld', 'correlation_regime_vol',
       'ADR_smooth', 'zbt', 'Pct_Above_MA50', 'Pct_Above_MA200',
       'NH_NL_Ratio_smooth', 'volum_surge'],
      dtype='object')

In [ ]:
import umap
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import (
    QuantileTransformer, PowerTransformer, MinMaxScaler,
    RobustScaler, MaxAbsScaler, StandardScaler
)
from sklearn.metrics import (
    silhouette_score, calinski_harabasz_score, davies_bouldin_score
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import itertools


# Define scalers
scalers = {
    "Quantile (uniform)": QuantileTransformer(output_distribution='uniform', random_state=0),
    "Power (Yeo-Johnson)": PowerTransformer(method='yeo-johnson'),
    "MinMax": MinMaxScaler(),
    "Robust": RobustScaler(),
    "MaxAbs": MaxAbsScaler(),
    "Standard": StandardScaler(),
}

# Define parameter grids
N_COMP = [3, 5, 8, 10]
N_NEIG = [8, 15, 30]
MIN_D = [0.01, 0.1, 0.2]
METRIC = ["correlation", "euclidean"]

# Store all results
all_results = []

# Range of K values to test
k_range = range(2, 8)

# Loop over scalers
for scaler_name, scaler in scalers.items():
    scaled = scaler.fit_transform(df_stationary)

    # Loop over all UMAP parameter combinations
    for n_comp, n_neig, min_d, metric in itertools.product(N_COMP, N_NEIG, MIN_D, METRIC):
        print(f"\n🔍 Scaler: {scaler_name} | UMAP: n_comp={n_comp}, n_neig={n_neig}, min_d={min_d}, metric={metric}")

        # Apply UMAP
        X_umap = umap.UMAP(
            n_neighbors=n_neig,
            min_dist=min_d,
            n_components=n_comp,
            metric=metric
        ).fit_transform(scaled)

        # Loop over cluster counts
        for k in k_range:
            # GMM
            gmm = GaussianMixture(n_components=k, random_state=0)
            gmm_labels = gmm.fit_predict(X_umap)

            all_results.append({
                "scaler": scaler_name,
                "method": "GMM",
                "k": k,
                "n_components": n_comp,
                "n_neighbors": n_neig,
                "min_dist": min_d,
                "metric": metric,
                "bic": gmm.bic(X_umap),
                "aic": gmm.aic(X_umap),
                "silhouette": silhouette_score(X_umap, gmm_labels),
                "calinski_harabasz": calinski_harabasz_score(X_umap, gmm_labels),
                "davies_bouldin": davies_bouldin_score(X_umap, gmm_labels)
            })

            # KMeans
            kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
            kmeans_labels = kmeans.fit_predict(X_umap)

            all_results.append({
                "scaler": scaler_name,
                "method": "KMeans",
                "k": k,
                "n_components": n_comp,
                "n_neighbors": n_neig,
                "min_dist": min_d,
                "metric": metric,
                "bic": None,
                "aic": None,
                "silhouette": silhouette_score(X_umap, kmeans_labels),
                "calinski_harabasz": calinski_harabasz_score(X_umap, kmeans_labels),
                "davies_bouldin": davies_bouldin_score(X_umap, kmeans_labels)
            })


🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.01, metric=correlation

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.01, metric=euclidean

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.1, metric=correlation

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.1, metric=euclidean

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.2, metric=correlation

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=8, min_d=0.2, metric=euclidean

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=15, min_d=0.01, metric=correlation

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=15, min_d=0.01, metric=euclidean

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=15, min_d=0.1, metric=correlation

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=15, min_d=0.1, metric=euclidean

🔍 Scaler: Quantile (uniform) | UMAP: n_comp=3, n_neig=15, min_d=0.2, metric=correlation

🔍 Scaler: Quantile (uniform) | U

In [87]:
results_df = pd.DataFrame(all_results)
results_df.to_csv("clustering_experiment_results.csv", index=False)